# 1. Creating, Reading and Writing
Pandas: 用于数据分析的python数据分析库
```python
import pandas as pd
```
数据结构包括dataframe和series
## 如何创建
dataframe相当于一个二维表格
DataFrame的构造器传入一个字典，字典的key值为列名，value值为对应列的entry列表
```python
pd.DataFrame({'Yes':[59,21], 'No': [131, 21]})
```
指定行标签进行构造
```python
pd.DataFrame({'Bob':['I liked it', 'It was awful'], 'Sue': ['Pretty good', 'Bland']}, index=['Product A', 'Product B'])
```

series相当于一个列表，可视为dataframe的一列
```python
pd.Series([1,2,3,4,5])
pd.Series([10, 20, 30], index=['2015', '2016', '2017'], name='Product A')
```
注意：Series只有一列，因而没有列名，而是有一个列表名

从文件中读取
.csv文件："Comma-Separated Values" file
```python
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv")
wine_reviews.shape
wine_reviews.head()
# 处理.csv文件的内嵌行索引
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
```

将dataframe写入文件
```python
data.to_csv("name.csv")
```

# 2. Indexing, Selecting and Assigning
在dataframe或series中选择特定值
## indexing
### 沿用的python语法
一个dataframe中的所有对象有着相同的结构，使用`.`访问对象的列或者说属性，使用字典索引的方式访问
举个例子
```python
reviews.country
reviews['country']
reviews['country'][0]
```

### pandas独有的索引方式
使用`iloc`方法或者`loc`方法
与沿用的python先列后行的索引方式不同，这种索引方式是先行后列
包含了两种selection方式
#### index-based selection
根据数据在dataframe中的索引位置来进行选择，使用`iloc`方法
```python
reviews.iloc[0]
reviews.iloc[:, 0]
```
#### label-based selection
使用dataframe中的索引值来进行选择，使用`loc`方法
```python
reviews.loc[0, 'country']
reviews.loc[:, ['taster_name', 'taster_twitter_handle', 'points']]
```
注意：
`iloc`方法的索引结果与沿用python相同，即`df.iloc[0:1000]`返回索引为0~999的结果
而`loc`方法的索引结果为`df.loc[0:1000]`返回0~1000的结果

修改索引以进行更好的label-based selection
```python
reviews.set_index("title")
```
#### conditional selection
```python
reviews.loc[reviews.country == 'Italy'] # 选择country属性为Italy的行
reviews.loc[(reviews.country == 'Italy') & (reviews.points >= 90)] # 两个限制条件的selection
```

**Pandas内置的条件筛选器**
```python
reviews.loc[reviews.country.isin(['Italy', 'France'])] # 将筛选值限制在一个list中，数值的话 直接><=
reviews.loc[reviews.price.notnull()] # 过滤空值
```
### 赋值
```python
reviews['critic'] = 'everyone'
reviews['index_backwards'] = range(len(reviews), 0, -1)
```

## 例题
从名为reviews的dataframe中筛选出所有评分不低于95且产地为Australia or New Zealand的评价
```python
top_oceania_wines = reviews.loc[(reviews.country.isin(['Australia', 'New Zealand']))&(reviews.points >= 95)]

```

# 3. Renaming and Combining
## Renaming
`rename()`方法：用来修改索引名或列名
举个例子
```python
reviews.rename(columns={'points': 'score'}) # 修改列名
reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})
```
`set_index()`方法：设置索引列
`rename_axis()`方法：修改行索引名或列索引名
```python
reviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns')
```
## Combining
三种方法：`concat()`方法、`join()`方法、`merge()`方法
当多个dataframe、series有相同的列名时，使用`concat()`方法进行拼接
举个例子
```python
canadian_youtube = pd.read_csv("../input/youtube-new/CAvideos.csv")
british_youtube = pd.read_csv("../input/youtube-new/GBvideos.csv")
pd.concat([canadian_youtube, british_youtube])
```
当多个dataframe有着相同的行索引时，使用`join()`方法进行拼接
举个例子
```python
left = canadian_youtube.set_index(['title', 'trending_date'])
right = british_youtube.set_index(['title', 'trending_date'])

left.join(right, lsuffix='_CAN', rsuffix='_UK') # 为了处理两个dataframe有相同列名的情况

# 另一个例子
powerlifting_meets = pd.read_csv("../input/powerlifting-database/meets.csv")
powerlifting_competitors = pd.read_csv("../input/powerlifting-database/openpowerlifting.csv")
# 将这两个dataframe通过同样的index进行拼接
powerlifting_combined = powerlifting_meets.set_index("MeetID").join(powerlifting_competitors.set_index("MeetID"))
```

# 4. Summary Functions and Maps
开始数据挖掘！
## Summary Functions
`describe()`方法
举个例子
```python
# 针对数值型数据
reviews.points.describe()
# 针对字符串型数据
reviews.taster_name.describe()
# 平均值
reviews.points.mean()
# 值列表
reviews.taster_name.unique()
# 值频率
reviews.taster_name.value_counts()
```
## Maps
映射，将一组数据映射到另一组数据，转换数据格式、为当前数据创建一个新的表示
两种映射方法
1. `map()`方法
2. `apply()`方法
3. 四则运算

**map()方法**
标准化数据
```python
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p-review_points_mean)
```

**apply()方法**
功能更加强大，可以针对不同列定制不同的映射方法
```python
def remean_points(row):
    row.points = row.points - reviews.points.mean()
    return row

reviews.apply(remean_points, axis='columns') # 这里的axis参数指明了映射的轴方向，如果为columns，表示处理每一行，即对每一个样本进行处理；如果为index，表示处理每一列，即对每一个特征进行处理
```

**注意**
`map()`方法和`apply()`方法都返回新的dataframe，而不会修改原有的数据

**四则运算符**
```python
# 标准化实现
review_points_mean = reviews.points.mean()
reviews.points - review_points_mean
# 字符串拼接
reviews.country + "-" + reviews.region_1
```
**说明**
使用四则运算符进行处理使用Pandas内置加速，会比`map()`方法和`apply()`方法都快
但`map()`方法和`apply()`方法比四则运算符更灵活

## 练习题
1. wine数据集中选择出`bargain_wine`，即性价比最高的
```python
bargain_idx = (reviews.points / reviews.price).idxmax()
bargain_wine = reviews.loc[bargain_idx, 'title']
```

2. 在reviews数据集中查找包含'tropical'或者'fruity'的数量
```python
n_trop = reviews.description.map(lambda desc: 'tropical' in desc).sum()
n_fruity = reviews.description.map(lambda desc: 'fruity' in desc).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])
```

3. 将reviews数据集中的points进行转换得到star数目
```python
def stars(row):
    if row.country == 'Canada':
        return 3
    elif row.points >= 95:
        return 3
    elif row.points >=85:
        return 2
    elif:
        return 1

star_ratings = reviews.apply(stars, axis = 'columns')
```

# 5. Grouping and Sorting
对数据集进行分组，对不同组的数据执行不同的操作
利用`groupby()`函数

## Groupwise analysis
`groupby()`函数的使用
```python
reviews.groupby('points').points.count() # 实现了value_count()方法
reviews.groupby('points').price.min() # 得到相同point组内的价格最低wine
reviews.groupby('winery').apply(lambda df: df.title.iloc[0]) # 取得一组wine中第一个的title
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()]) # 在一个省内最好的wine
reviews.groupby('country').price.agg([len, min, max]) # 对数据集的每一个分组同时使用多个统计函数
```

## Multi-indexes
通过`groupby()`函数的操作，我们会得到一个拥有多个index的dataframe
```python
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
mi = countries_reviewed.index
type(mi) # 获得multi-index的dataframe的index
countries_reviewed.reset_index() # 将多index数据集恢复为单index数据
```

## Sorting
```python
# 指定排序指标的函数
countries_reviewed.sort_values(by = 'len') # 默认升序
countries_reviewed.sort_values(by = 'len', ascending=False) # 修改为降序
countries_reviewed.sort_index() # 按index列进行排序
countries_reviewed.sort_values(by=['country', 'len']) # 按多列values进行排序
```

## 典型例题
1. 多个排序条件的sort_values()方法，先按照min降序，再按照max降序
```python
sorted_varieties = price_extremes_sort_values(by = ['min', 'max'], ascending=False)
```
2. 使用multi-index的分组，再根据wine数量进行降序排列
```python
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)
```

**注意**
`data.count()`方法和`data.size()`方法的区别：`count()`方法返回一个dataframe，统计每个分组所有列的飞缺失值计数；而`size()`方法返回每个分组中的总列数，返回一个Series`

# 6. Data Types and Missing Values
## Dtypes
在dataframe或Series中的列数据类型被称为dtype
```python
# 得到某一个列的类型
reviews.price.dtype
# 得到dataframe中每列的数据类型
reviews.dtypes
# 字符串列的返回类型为object类型
# 数据类型转换
reviews.points.astype('float64')
# 索引类型
reviews.index.dtype
```

## Missing data
缺失值为`NaN`，即"Not a Number"，在数据存储过程中`NaN`为`float64`类型
Pandas中与缺失值相关的函数：`isnull()`方法、`notnull()`方法
```python
reviews[pd.isnull(reviews.country)] # 得到country列为NaN的数据行
```
缺失值填充
`fillna()`方法
```python
reviews.region_2.fillna("Unknown")
# backfill strategy
reviews.region_2.fillna(method='bfill', inplace=True)
# 替换策略
reviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")
```

## 典型例题
1. 计数reviews数据集中有缺失值的price列
```python
n_missing_prices = reviews.price.isnull().sum() # dataframe的每一列都有isnull()方法

n_missing_prices = pd.isnull(reviews.price).sum()
```

2. 缺失值填充，计数各个region_1的数目，降序排列
```python
reviews_per_region = reviews.region_1.fillna('Unknown').value_counts().sort_values(ascending=False)
```